In [1]:
# Libraries
import random
import numpy as np
import time

In [2]:
# Parameters
generations = 100 # number of epochs/generations
numIndividuals = 30 # population size
k = 5 # tournament size (determines the selection pressure)
mr = 0.02 # mutation rate
rr = 1 # reproduction rate (= 1 - crossover rate)

# Genetic Algorithm
def geneticAlgorithm(items, capacity, numI, solution):
    
    initialPopulation = generateInitialPopulation(items, numI)
    population = [(bits, fitness(items, bits, capacity)) for bits in initialPopulation]
    #print(population)
    
    avg_fitnesses = []
    for i in range(generations):
        #print(f"Average fitness: {average_fitness(population, solution):.3f} ", end="\r")
        #time.sleep(0.5)
        population = nextGeneration(population, items, capacity)
    
    
    population = sorted(population, key=lambda i: i[1], reverse=True)
    #print(population[0][1])
    return (solution-population[0][1])/solution

# Generate initial population
def generateInitialPopulation(items, numI):
    
    population = []
    
    while len(population) != numIndividuals:
        
        bits = [ random.choice([0, 1]) for _ in items ]
        population.append(bits)

    return population

# Calculate fitness
def fitness(items, bits, capacity):
    
    weight = sum([bit * item[0] for item, bit in zip(items, bits)])
    if weight > capacity:
        return 0

    value = sum([ bit * item[1] for item, bit in zip(items, bits)])
    return value

# Selection with tournament 
def selection(population):
    
    parents = []
    
    random.shuffle(population)
    copy = population.copy()
    
    # Select individuals
    for _ in range(int(0.6*numIndividuals)):
        
        # Shuffle the list to have random picks
        random.shuffle(copy)
        
        # Take k random individuals and select the best one
        tempList = []
        tempList = copy[0:k]
        tempList = sorted(tempList, key=lambda i: i[1], reverse=True)
        copy.remove(tempList[0])
        parents.append(tempList[0])
            
    return parents

# Crossover from parents
def crossover(parents, items, capacity):
    N = len(items)
    
    bits1 = parents[0][0][:N//2] + parents[1][0][N//2:]
    bits2 = parents[0][0][N//2:] + parents[1][0][:N//2]
    child1 = (bits1, fitness(items, bits1, capacity))
    child2 = (bits2, fitness(items, bits2, capacity))
    
    return [child1, child2]

# Mutation
def mutate(population):
    
    for individual in population:
        for i in range(len(individual[0])):
            if random.random() < mr:
                # Random bit
                bit = random.randint(0,len(individual[0])-1)
                # Flip it
                if individual[0][bit]==0:
                    individual[0][bit] = 1
                else:
                    individual[0][bit] = 0
                

# Generate next generation              
def nextGeneration(population, items, capacity):
    next_gen = []
    
    # we run selection and get parents
    parents = selection(population)
    
    while len(next_gen) < len(population):
        
        random.shuffle(parents)
        tempP = [parents[0], parents[1]]
        children = []
        
        # reproduction
        if random.random() < rr:
            children = tempP
        else:
            # crossover
            children = crossover(tempP, items, capacity)

        # mutation
        mutate(children)

        next_gen.extend(children)

    return next_gen

# Average fitness
def average_fitness(population, solution):
    
    sumV = 0
    for p in population:
        sumV += (solution - p[1])/solution
    
    return sumV/len(population)


# Recursive Dynamic programming
def dynamicProgramming(capacity, weights, costs, numI):
    K = [[0 for x in range(capacity + 1)] for x in range(numI + 1)]
 
    # Build table K[][] in bottom up manner
    for i in range(numI + 1):
        for w in range(capacity + 1):
            if i == 0 or w == 0:
                K[i][w] = 0
            elif weights[i-1] <= w:
                K[i][w] = max(costs[i-1] + K[i-1][w-weights[i-1]], K[i-1][w])
            else:
                K[i][w] = K[i-1][w]
 
    return K[numI][capacity]

In [3]:
# importing data
with open('data.txt') as f:
    lines = f.readlines()

#importing solution
solList = []
solDict = {}

#calculate solution
for l in lines:
    data = l.split()
    idR = data[0]
    num = int(data[1])
    capacity = int(data[2])
    items = []
    wt = []
    val = []
    
    for i in range(3, 3+(num*2) ,2):
        items.append((int(data[i]), int(data[i+1])))
        wt.append(int(data[i]))
        val.append(int(data[i+1]))
                
    solDict[data[0]] = dynamicProgramming(capacity, wt, val, num)

In [4]:
errorList = []
st = time.time_ns()   
for line in lines:
    
    # for every line I solve the knapsack problem
    data = line.split()
    
    # id of the instance
    idR = data[0]
    
    # number of items
    num = int(data[1])
    
    # capacity of knapsack
    capacity = int(data[2])
    
    # available items
    items = []
    wt = []
    val = []
    
    for i in range(3, 3+(num*2) ,2):
        items.append((int(data[i]), int(data[i+1])))
        wt.append(int(data[i]))
        val.append(int(data[i+1]))
                
    print(f"Computing instance n°{idR}", end="\r")
    
    errorList.append((geneticAlgorithm(items, capacity, len(items), solDict[idR])))
    
    
et = time.time_ns()  
    

In [5]:
# Statistcs
print("Runtime: " + str(round((et - st)/60000000000,4)) + " minutes")
print("Average error: " + str(round(sum(errorList)*100/len(errorList),2)) + " %")

Runtime: 0.4543 minutes
Average error: 7.71 %
